In [1]:
import os
import glob
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Embedding, LSTM
from keras.layers import Permute, RepeatVector, Concatenate, Activation
from keras.layers import dot
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import seaborn as sns

Using TensorFlow backend


In [2]:
import csv
train_gt = []
with open('gt_split/train.tsv','r') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        if len(line)==2:
            path, exp = line[0], line[1]
            train_gt.append([path, exp])

test_gt = []
with open('gt_split/validation.tsv','r') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        if len(line)==2:
            path, exp = line[0], line[1]
            test_gt.append([path, exp])

FileNotFoundError: [Errno 2] No such file or directory: 'gt_split/train.tsv'

In [24]:
import pandas as pd
train_gt = pd.read_csv('gt_split/train.tsv', sep='\t', names=['path', 'gt'])

In [35]:
train_gt

[['KAIST/TrainData2_7_sub_33', '\\log_{2} 8 + \\log_{3} 9 + \\log_{4} 1 6'],
 ['expressmatch/76_leissi', 'a = b \\cos C + c \\cos B'],
 ['MathBrush/200926-131-183', ' 0 '],
 ['MathBrush/200922-1017-104', ' 4.7 '],
 ['expressmatch/70_carlos', '\\cos \\theta = \\frac{x}{\\sqrt{x^2 + y^2}}'],
 ['MathBrush/2009212-1031-72', ' { - 6 } '],
 ['MathBrush/200923-1553-242', ' { j } _ { \\mbox { y } } '],
 ['MathBrush/200923-1251-144',
  ' { { \\mbox { P } - \\sqrt { 4 } } \\lt \\left [ \\mbox { k } \\right ] } '],
 ['MathBrush/2009212-1031-137', ' \\mbox { b } '],
 ['HAMEX/formulaire010-equation016', 'x^5=x+1'],
 ['HAMEX/formulaire023-equation061', '\\gamma(0) = x'],
 ['MathBrush/2009213-137-164',
  ' { \\mbox { E } + \\mbox { T } \\left ( S \\right ) } '],
 ['KAIST/TrainData2_5_sub_95',
  '\\sqrt{1 + \\sqrt{2 + \\sqrt{3 + \\sqrt{4}}}}'],
 ['MathBrush/200926-131-228', ' { \\mbox { B } { \\mbox { P } z } } '],
 ['MfrDB/MfrDB0463', '\\int\\limits_{0}^{\\infty} d x x'],
 ['HAMEX/formulaire025-equat

In [ ]:
# Define the Image Encoder
image_input = Input(shape=(image_width, image_height, num_channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(max_pool2)
encoder_dense = Dense(256, activation='relu')(flatten)

# Define the LaTeX Decoder
latex_input = Input(shape=(max_latex_length,))
latex_embedding = Embedding(output_vocab_size, embedding_dim)(latex_input)
decoder_lstm = LSTM(256, return_sequences=True)(latex_embedding)

# Attention Mechanism
attn_repeat = RepeatVector(max_latex_length)(encoder_dense)
attn_concat = Concatenate(axis=-1)([decoder_lstm, attn_repeat])
attn_dense1 = Dense(256, activation='tanh')(attn_concat)
attn_score = Dense(1, activation='softmax')(attn_dense1)
attn_dot = dot([attn_score, encoder_dense], axes=(1, 1))
attn_dot = Permute((2, 1))(attn_dot)
context = Concatenate(axis=-1)([decoder_lstm, attn_dot])
decoder_attn = Activation('tanh')(context)

latex_output = Dense(output_vocab_size, activation='softmax')(decoder_attn)

# Connect the Encoder and Decoder
image_to_latex_model = Model(inputs=[image_input, latex_input], outputs=latex_output)

# Compile the Model
image_to_latex_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Summary of the Model
image_to_latex_model.summary()
